## Use boto3 para entrenar y actualizar un SageMaker Endpoint existente con un modelo de bosque aleatorio de Scikit-Learn recién entrenado

En este cuaderno, mostramos cómo usar Amazon SageMaker para desarrollar, entrenar e implementar un modelo de ML basado en Scikit-Learn (Random Forest). Luego mostramos cómo actualizar un SageMaker Endpoint existente con un modelo recién entrenado, <b>mientras no haya pérdida de disponibilidad</b>.

Esto se hace usando `boto3`, un cliente de bajo nivel que representa el servicio Amazon SageMaker. El uso de `boto3` es muy útil cuando se usan los servicios de SageMaker desde otros recursos que no sean portátiles, como las funciones de Lambda, Airflow o Jenkins.

Ejecutará los siguientes pasos usando `boto3`:
 - Preparar los datos de entrenamiento/prueba y escribir un script de modo de secuencia de comandos.
 - Lanzar el 1er trabajo de entrenamiento.
 - Implemente el primer modelo en un punto final de SageMaker y realice pocas solicitudes de inferencia.
 - Lanzar el segundo trabajo de entrenamiento.
 - Actualice SageMaker Endpoint con el segundo modelo y realice pocas solicitudes de inferencia.
 - Limpieza opcional.


Puede encontrar más información sobre boto3 en la [página de documentación de Boto3 SageMaker] (https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#client).

Puede encontrar más información sobre Scikit-Learn en la [página de documentación de Scikit-Learn] (https://scikit-learn.org/stable/index.html).

Usamos el [conjunto de datos de vivienda de California](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html), presente en Scikit-Learn.

Más información sobre el conjunto de datos:

Este conjunto de datos se obtuvo del repositorio `StatLib`. http://lib.stat.cmu.edu/datasets/

La variable objetivo es el valor medio de la vivienda para los distritos de California.

Este conjunto de datos se derivó del censo de EE. UU. de 1990, utilizando una fila por grupo de bloques censales. Un grupo de bloques es la unidad geográfica más pequeña para la que la Oficina del Censo de EE. UU. publica datos de muestra (un grupo de bloques suele tener una población de 600 a 3000 personas).

Se puede descargar/cargar usando la función `sklearn.datasets.fetch_california_housing`.
 

## Importar bibliotecas de Python

Ahora importa bibliotecas de Python como `sklearn`, `pandas`, `numpy` y `boto3`.

También importamos `sagemaker`, que es el SDK de SageMaker Python de alto nivel. Esto tiene como objetivo obtener el depósito y la función de ejecución predeterminados de SageMaker. Aparte de eso, toda la funcionalidad de SageMaker se demostrará usando `boto3`.

In [4]:
import datetime
import time
import tarfile

import boto3
import pandas as pd
import numpy as np
from sagemaker import get_execution_role
import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing


sm_boto3 = boto3.client("sagemaker")

sess = sagemaker.Session()

region = sess.boto_session.region_name

bucket = sess.default_bucket()  # this could also be a hard-coded bucket name

print("Using bucket " + bucket)

Using bucket sagemaker-us-east-1-808640880671


## Preparar datos
Cargamos un conjunto de datos de `sklearn`, lo dividimos y lo enviamos a S3

In [5]:
# usamos el conjunto de datos de vivienda de California
data = fetch_california_housing()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.25, random_state=42
)

trainX = pd.DataFrame(X_train, columns=data.feature_names)
trainX["target"] = y_train

testX = pd.DataFrame(X_test, columns=data.feature_names)
testX["target"] = y_test

In [7]:
trainX.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,4.2143,37.0,5.288235,0.973529,860.0,2.529412,33.81,-118.12,2.285
1,5.3468,42.0,6.364322,1.087940,957.0,2.404523,37.16,-121.98,2.799
2,3.9191,36.0,6.110063,1.059748,711.0,2.235849,38.45,-122.69,1.830
3,6.3703,32.0,6.000000,0.990196,1159.0,2.272549,34.16,-118.41,4.658
4,2.3684,17.0,4.795858,1.035503,706.0,2.088757,38.57,-121.33,1.500


In [9]:
trainX.to_csv("california_train.csv")
testX.to_csv("california_test.csv")

In [10]:
# enviar datos a S3. SageMaker tomará 
# datos de entrenamiento de s3
trainpath = sess.upload_data(
    path="california_train.csv", bucket=bucket, key_prefix="sagemaker/sklearn-california"
)

testpath = sess.upload_data(
    path="california_test.csv", bucket=bucket, key_prefix="sagemaker/sklearn-california"
)

## Escribiendo codigo en *Modo Script*
La siguiente secuencia de comandos contiene funciones de capacitación e inferencia y puede ejecutarse tanto en el hardware de capacitación de SageMaker como localmente (escritorio, computadora portátil SageMaker, en las instalaciones, etc.). Puede encontrar instrucciones detalladas en la [página de documentación del SDK de SageMaker Python de Scikit-learn](https://sagemaker.readthedocs.io/en/stable/using_sklearn.html#preparing-the-scikit-learn-training-script).

In [11]:
%%writefile script.py

import argparse
import joblib
import os

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor


# inference functions ---------------
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf


if __name__ == "__main__":

    print("extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    # to simplify the demo we don't use all sklearn RandomForest hyperparameters
    parser.add_argument("--n-estimators", type=int, default=10)
    parser.add_argument("--min-samples-leaf", type=int, default=3)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="california_train.csv")
    parser.add_argument("--test-file", type=str, default="california_test.csv")
    parser.add_argument(
        "--features", type=str
    )  # in this script we ask user to explicitly name features
    parser.add_argument(
        "--target", type=str
    )  # in this script we ask user to explicitly name the target

    args, _ = parser.parse_known_args()

    print("reading data")
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    print("building training and testing datasets")
    X_train = train_df[args.features.split()]
    X_test = test_df[args.features.split()]
    y_train = train_df[args.target]
    y_test = test_df[args.target]

    # train
    print("training model")
    model = RandomForestRegressor(
        n_estimators=args.n_estimators, min_samples_leaf=args.min_samples_leaf, n_jobs=-1
    )

    model.fit(X_train, y_train)

    # print abs error
    print("validating model")
    abs_err = np.abs(model.predict(X_test) - y_test)

    # print couple perf metrics
    for q in [10, 50, 90]:
        print("AE-at-" + str(q) + "th-percentile: " + str(np.percentile(a=abs_err, q=q)))

    # persist model
    path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, path)
    print("model persisted at " + path)
    print(args.min_samples_leaf)

Writing script.py


## Entrenamiento de SageMaker
### Lanzamiento de un entrenamiento con `boto3`
`boto3` es más detallado pero brinda más visibilidad en los detalles de bajo nivel de Amazon SageMaker

In [13]:
# primero comprima el código y envíelo a S3

source = "source.tar.gz"
project = "scikitlearn-california-train-from-boto3"

tar = tarfile.open(source, "w:gz")
tar.add("script.py")
tar.close()

s3 = boto3.client("s3")
s3.upload_file(source, bucket, project + "/" + source)

Cuando usamos `boto3` para iniciar un trabajo de entrenamiento, debemos apuntar explícitamente a una imagen de Docker.

In [14]:
from sagemaker import image_uris

FRAMEWORK_VERSION = "0.23-1"

training_image = image_uris.retrieve(
    framework="sklearn",
    region=region,
    version=FRAMEWORK_VERSION,
    py_version="py3",
    instance_type="ml.c5.xlarge",
)
print(training_image)

683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3


## Lanzar el primer trabajo de entrenamiento

Esto iniciará un trabajo de entrenamiento modelo. Una vez que finaliza el entrenamiento, Amazon SageMaker guarda los artefactos del modelo resultante en una ubicación de Amazon S3 que especifique.

Si elige alojar su modelo mediante los servicios de alojamiento de Amazon SageMaker, puede utilizar los artefactos del modelo resultante como parte del modelo. También puede usar los artefactos en un servicio de aprendizaje automático que no sea Amazon SageMaker, siempre que sepa cómo usarlos para la inferencia.

Puede encontrar más información sobre `create_training_job` en la [página de documentación de Boto3 SageMaker] (https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_training_job) .

In [15]:
training_job_1_name = "sklearn-boto3-1-" + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

training_job_1_response = sm_boto3.create_training_job(
    TrainingJobName=training_job_1_name,
    HyperParameters={
        "n_estimators": "300",
        "min_samples_leaf": "3",
        "sagemaker_program": "script.py",
        "features": "MedInc HouseAge AveRooms AveBedrms Population AveOccup Latitude Longitude",
        "target": "target",
        "sagemaker_submit_directory": "s3://" + bucket + "/" + project + "/" + source,
    },
    AlgorithmSpecification={
        "TrainingImage": training_image,
        "TrainingInputMode": "File",
        "MetricDefinitions": [
            {"Name": "median-AE", "Regex": "AE-at-50th-percentile: ([0-9.]+).*$"},
        ],
    },
    RoleArn=get_execution_role(),
    InputDataConfig=[
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": trainpath,
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
        },
        {
            "ChannelName": "test",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": testpath,
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
        },
    ],
    OutputDataConfig={"S3OutputPath": "s3://" + bucket + "/sagemaker-sklearn-artifact/"},
    ResourceConfig={"InstanceType": "ml.c5.xlarge", "InstanceCount": 1, "VolumeSizeInGB": 10},
    StoppingCondition={"MaxRuntimeInSeconds": 86400},
    EnableNetworkIsolation=False,
)

training_job_1_response

{'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:808640880671:training-job/sklearn-boto3-1-2022-05-04-13-19-28',
 'ResponseMetadata': {'RequestId': '8dcada3a-a618-4383-aa22-4e0ea01a0205',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8dcada3a-a618-4383-aa22-4e0ea01a0205',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '110',
   'date': 'Wed, 04 May 2022 13:19:29 GMT'},
  'RetryAttempts': 0}}

### Espere a que finalice el primer trabajo de entrenamiento

In [18]:
client = boto3.client("sagemaker")

training_job_1_details = client.describe_training_job(TrainingJobName=training_job_1_name)
print(training_job_1_details["TrainingJobStatus"])

while training_job_1_details["TrainingJobStatus"] == "InProgress":
    training_job_1_details = client.describe_training_job(TrainingJobName=training_job_1_name)
    time.sleep(15)

print(training_job_1_details["TrainingJobStatus"])
training_job_1_details

Completed
Completed


{'TrainingJobName': 'sklearn-boto3-1-2022-05-04-13-19-28',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:808640880671:training-job/sklearn-boto3-1-2022-05-04-13-19-28',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-808640880671/sagemaker-sklearn-artifact/sklearn-boto3-1-2022-05-04-13-19-28/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'features': 'MedInc HouseAge AveRooms AveBedrms Population AveOccup Latitude Longitude',
  'min_samples_leaf': '3',
  'n_estimators': '300',
  'sagemaker_program': 'script.py',
  'sagemaker_submit_directory': 's3://sagemaker-us-east-1-808640880671/scikitlearn-california-train-from-boto3/source.tar.gz',
  'target': 'target'},
 'AlgorithmSpecification': {'TrainingImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3',
  'TrainingInputMode': 'File',
  'MetricDefinitions': [{'Name': 'median-AE',
    'Regex': 'AE-at-50th-percentile: ([0

## Crear un modelo para el primer trabajo de entrenamiento

Esto creará un modelo en Amazon SageMaker. En la solicitud, nombra el modelo y describe un contenedor principal. Para el contenedor principal, especifica la imagen de Docker que contiene el código de inferencia, los artefactos (del entrenamiento anterior) y un mapa de entorno personalizado que usa el código de inferencia cuando implementa el modelo para las predicciones.

Puede encontrar más información sobre `create_model` en la [página de documentación de Boto3 SageMaker](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model) .

In [19]:
model_1_name = "sklearn-model-1-" + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

create_model_1_response = client.create_model(
    ModelName=model_1_name,
    PrimaryContainer={
        "Image": training_job_1_details["AlgorithmSpecification"]["TrainingImage"],
        "Mode": "SingleModel",
        "ModelDataUrl": training_job_1_details["ModelArtifacts"]["S3ModelArtifacts"],
        "Environment": {
            "SAGEMAKER_CONTAINER_LOG_LEVEL": "20",
            "SAGEMAKER_PROGRAM": training_job_1_details["HyperParameters"]["sagemaker_program"],
            "SAGEMAKER_REGION": region,
            "SAGEMAKER_SUBMIT_DIRECTORY": training_job_1_details["HyperParameters"][
                "sagemaker_submit_directory"
            ],
        },
    },
    ExecutionRoleArn=get_execution_role(),
)

create_model_1_response

{'ModelArn': 'arn:aws:sagemaker:us-east-1:808640880671:model/sklearn-model-1-2022-05-04-13-25-18',
 'ResponseMetadata': {'RequestId': '19cd4e35-437e-40ab-86a9-8660e33aa14b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '19cd4e35-437e-40ab-86a9-8660e33aa14b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '97',
   'date': 'Wed, 04 May 2022 13:25:17 GMT'},
  'RetryAttempts': 0}}

Ahora describirá el modelo que creó utilizando la API `describe_model`.

Puede encontrar más información sobre `describe_model` en la [página de documentación de Boto3 SageMaker](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.describe_model) .

In [20]:
client.describe_model(ModelName=model_1_name)

{'ModelName': 'sklearn-model-1-2022-05-04-13-25-18',
 'PrimaryContainer': {'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3',
  'Mode': 'SingleModel',
  'ModelDataUrl': 's3://sagemaker-us-east-1-808640880671/sagemaker-sklearn-artifact/sklearn-boto3-1-2022-05-04-13-19-28/output/model.tar.gz',
  'Environment': {'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
   'SAGEMAKER_PROGRAM': 'script.py',
   'SAGEMAKER_REGION': 'us-east-1',
   'SAGEMAKER_SUBMIT_DIRECTORY': 's3://sagemaker-us-east-1-808640880671/scikitlearn-california-train-from-boto3/source.tar.gz'}},
 'ExecutionRoleArn': 'arn:aws:iam::808640880671:role/service-role/AmazonSageMaker-ExecutionRole-20220503T130588',
 'CreationTime': datetime.datetime(2022, 5, 4, 13, 25, 18, 682000, tzinfo=tzlocal()),
 'ModelArn': 'arn:aws:sagemaker:us-east-1:808640880671:model/sklearn-model-1-2022-05-04-13-25-18',
 'EnableNetworkIsolation': False,
 'ResponseMetadata': {'RequestId': 'a36d49f5-5689-445c-bbc7-bdf834652f

## Crear una configuración de endpoint desde el primer modelo

Esto creará una configuración de punto de enlace que los servicios de alojamiento de Amazon SageMaker utilizan para implementar modelos. En la configuración, identifica uno o más modelos, creados con la API `CreateModel`, para implementar y los recursos que desea que Amazon SageMaker aprovisione. Luego llamas a la API `CreateEndpoint`.

Puede encontrar más información sobre `create_endpoint_config` en la [página de documentación de Boto3 SageMaker](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint_config) .

In [21]:
endpoint_config_1_name = "sklearn-endpoint-config-1-" + datetime.datetime.now().strftime(
    "%Y-%m-%d-%H-%M-%S"
)

endpoint_config_1_response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_1_name,
    ProductionVariants=[
        {
            "VariantName": "AllTrafficVariant",
            "ModelName": model_1_name,
            "InitialInstanceCount": 1,
            "InstanceType": "ml.c5.large",
            "InitialVariantWeight": 1,
        },
    ],
)

endpoint_config_1_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:808640880671:endpoint-config/sklearn-endpoint-config-1-2022-05-04-13-28-46',
 'ResponseMetadata': {'RequestId': '0863719e-ad4d-40eb-bc04-e5cc88ee1a8b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0863719e-ad4d-40eb-bc04-e5cc88ee1a8b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '126',
   'date': 'Wed, 04 May 2022 13:28:46 GMT'},
  'RetryAttempts': 0}}

## Implemente la primera configuración del endpoint en un endpoint en tiempo real

Esto creará un punto final utilizando la configuración de punto final especificada en la solicitud. Amazon SageMaker utiliza el punto de enlace para aprovisionar recursos e implementar modelos. Tenga en cuenta que ya ha creado la configuración del punto final con la API `CreateEndpointConfig` en el paso anterior.

Puede encontrar más información sobre `create_endpoint` en la [página de documentación de Boto3 SageMaker](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint) .

In [22]:
endpoint_name = "sklearn-endpoint-" + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_1_name,
)

create_endpoint_response

{'EndpointArn': 'arn:aws:sagemaker:us-east-1:808640880671:endpoint/sklearn-endpoint-2022-05-04-13-29-56',
 'ResponseMetadata': {'RequestId': 'ff277040-f86d-4fc3-9991-8ee45ad4ec06',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ff277040-f86d-4fc3-9991-8ee45ad4ec06',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '104',
   'date': 'Wed, 04 May 2022 13:29:56 GMT'},
  'RetryAttempts': 0}}

### Espere a que Endpoint esté listo

In [27]:
describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
print(describe_endpoint_response["EndpointStatus"] + "...")

while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    time.sleep(15)

print(describe_endpoint_response["EndpointStatus"])
describe_endpoint_response

InService...
InService


{'EndpointName': 'sklearn-endpoint-2022-05-04-13-29-56',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:808640880671:endpoint/sklearn-endpoint-2022-05-04-13-29-56',
 'EndpointConfigName': 'sklearn-endpoint-config-1-2022-05-04-13-28-46',
 'ProductionVariants': [{'VariantName': 'AllTrafficVariant',
   'DeployedImages': [{'SpecifiedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3',
     'ResolvedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn@sha256:1b1a557ceb336c12d1f8ade5f0cd79781a8df07dde3128544dfef6679feb261e',
     'ResolutionTime': datetime.datetime(2022, 5, 4, 13, 29, 57, 717000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2022, 5, 4, 13, 29, 57, 80000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 5, 4, 13, 32, 17, 324000, tzinfo=tzlocal()),

## Invocar Endpoint con `boto3`

Después de implementar un modelo en producción mediante los servicios de alojamiento de Amazon SageMaker, sus aplicaciones cliente utilizan esta API para obtener inferencias del modelo alojado en el punto de enlace especificado.

Para obtener una descripción general de Amazon SageMaker, [consulte Cómo funciona](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works.html).

Amazon SageMaker elimina todos los encabezados POST, excepto los que admite la API. Amazon SageMaker podría agregar encabezados adicionales. No debe confiar en el comportamiento de los encabezados fuera de los enumerados en la sintaxis de la solicitud.

Las llamadas a `InvokeEndpoint` se autentican mediante AWS Signature Version 4. Para obtener más información, consulte Solicitudes de autenticación (AWS Signature Version 4) en la Referencia de la API de Amazon S3.

Los contenedores modelo de un cliente deben responder a las solicitudes en un plazo de 60 segundos. El modelo en sí puede tener un tiempo máximo de procesamiento de 60 segundos antes de responder a las invocaciones. Si su modelo va a tardar entre 50 y 60 segundos de tiempo de procesamiento, el tiempo de espera del zócalo del SDK debe establecerse en 70 segundos.

Puede encontrar más información sobre `invoke_endpoint` en la [página de documentación de Boto3 SageMakerRuntime](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime.html#SageMakerRuntime.Client. invocar_punto final).

In [28]:
runtime = boto3.client("sagemaker-runtime")

In [38]:
# csv serialization
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=testX[data.feature_names].to_csv(header=False, index=False).encode("utf-8"),
    ContentType="text/csv",
)

print(response["Body"].read()[:500])

b'[0.5246552380952382, 0.7116836940836941, 4.884692129545455, 2.553313333333333, 2.196873214285714, 1.6397502380952382, 2.303477359307359, 1.6989849206349206, 2.888873, 4.779673844444446, 1.2125477777777778, 2.016068452380952, 1.5302851370851374, 1.9184233333333336, 2.54005, 1.58476, 1.9199190476190473, 1.6213527777777776, 1.440369285714286, 0.976126111111111, 3.987773273809524, 3.4530853511904764, 1.6839254761904763, 3.9077470833333336, 1.7549811904761907, 0.548842619047619, 1.587321031746032, 0.'


## Inicie el segundo trabajo de entrenamiento

Esto iniciará el segundo trabajo de entrenamiento del modelo. Una vez completada la capacitación, creará un modelo, una configuración de punto final y actualizará el punto final existente con la configuración de punto final recién creada.

In [39]:
training_job_2_name = "sklearn-boto3-2-" + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

training_job_2_response = sm_boto3.create_training_job(
    TrainingJobName=training_job_2_name,
    HyperParameters={
        "n_estimators": "300",
        "min_samples_leaf": "3",
        "sagemaker_program": "script.py",
        "features": "MedInc HouseAge AveRooms AveBedrms Population AveOccup Latitude Longitude",
        "target": "target",
        "sagemaker_submit_directory": "s3://" + bucket + "/" + project + "/" + source,
    },
    AlgorithmSpecification={
        "TrainingImage": training_image,
        "TrainingInputMode": "File",
        "MetricDefinitions": [
            {"Name": "median-AE", "Regex": "AE-at-50th-percentile: ([0-9.]+).*$"},
        ],
    },
    RoleArn=get_execution_role(),
    InputDataConfig=[
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": trainpath,
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
        },
        {
            "ChannelName": "test",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": testpath,
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
        },
    ],
    OutputDataConfig={"S3OutputPath": "s3://" + bucket + "/sagemaker-sklearn-artifact/"},
    ResourceConfig={"InstanceType": "ml.c5.xlarge", "InstanceCount": 1, "VolumeSizeInGB": 10},
    StoppingCondition={"MaxRuntimeInSeconds": 86400},
    EnableNetworkIsolation=False,
)

training_job_2_response

{'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:808640880671:training-job/sklearn-boto3-2-2022-05-04-13-37-15',
 'ResponseMetadata': {'RequestId': '662ff5f4-1d79-4d59-8f36-25a51a166cb3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '662ff5f4-1d79-4d59-8f36-25a51a166cb3',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '110',
   'date': 'Wed, 04 May 2022 13:37:14 GMT'},
  'RetryAttempts': 0}}

### Espere a que finalice el segundo trabajo de entrenamiento

In [41]:
import boto3
import time

client = boto3.client("sagemaker")

training_job_2_details = client.describe_training_job(TrainingJobName=training_job_2_name)
print(training_job_2_details["TrainingJobStatus"] + "...")

while training_job_2_details["TrainingJobStatus"] == "InProgress":
    training_job_2_details = client.describe_training_job(TrainingJobName=training_job_2_name)
    time.sleep(15)

print(training_job_2_details["TrainingJobStatus"])
training_job_2_details

InProgress...
Completed


{'TrainingJobName': 'sklearn-boto3-2-2022-05-04-13-37-15',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:808640880671:training-job/sklearn-boto3-2-2022-05-04-13-37-15',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-808640880671/sagemaker-sklearn-artifact/sklearn-boto3-2-2022-05-04-13-37-15/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'features': 'MedInc HouseAge AveRooms AveBedrms Population AveOccup Latitude Longitude',
  'min_samples_leaf': '3',
  'n_estimators': '300',
  'sagemaker_program': 'script.py',
  'sagemaker_submit_directory': 's3://sagemaker-us-east-1-808640880671/scikitlearn-california-train-from-boto3/source.tar.gz',
  'target': 'target'},
 'AlgorithmSpecification': {'TrainingImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3',
  'TrainingInputMode': 'File',
  'MetricDefinitions': [{'Name': 'median-AE',
    'Regex': 'AE-at-50th-percentile: ([0

## Crear un modelo para el segundo trabajo de entrenamiento

In [42]:
model_2_name = "sklearn-model-2-" + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

create_model_2_response = client.create_model(
    ModelName=model_2_name,
    PrimaryContainer={
        "Image": training_job_2_details["AlgorithmSpecification"]["TrainingImage"],
        "Mode": "SingleModel",
        "ModelDataUrl": training_job_2_details["ModelArtifacts"]["S3ModelArtifacts"],
        "Environment": {
            "SAGEMAKER_CONTAINER_LOG_LEVEL": "20",
            "SAGEMAKER_PROGRAM": training_job_2_details["HyperParameters"]["sagemaker_program"],
            "SAGEMAKER_REGION": region,
            "SAGEMAKER_SUBMIT_DIRECTORY": training_job_2_details["HyperParameters"][
                "sagemaker_submit_directory"
            ],
        },
    },
    ExecutionRoleArn=get_execution_role(),
)

create_model_2_response

{'ModelArn': 'arn:aws:sagemaker:us-east-1:808640880671:model/sklearn-model-2-2022-05-04-13-41-46',
 'ResponseMetadata': {'RequestId': '490cb53b-d5be-462d-a8ae-11a902604d16',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '490cb53b-d5be-462d-a8ae-11a902604d16',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '97',
   'date': 'Wed, 04 May 2022 13:41:46 GMT'},
  'RetryAttempts': 0}}

In [43]:
client.describe_model(ModelName=model_2_name)

{'ModelName': 'sklearn-model-2-2022-05-04-13-41-46',
 'PrimaryContainer': {'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3',
  'Mode': 'SingleModel',
  'ModelDataUrl': 's3://sagemaker-us-east-1-808640880671/sagemaker-sklearn-artifact/sklearn-boto3-2-2022-05-04-13-37-15/output/model.tar.gz',
  'Environment': {'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
   'SAGEMAKER_PROGRAM': 'script.py',
   'SAGEMAKER_REGION': 'us-east-1',
   'SAGEMAKER_SUBMIT_DIRECTORY': 's3://sagemaker-us-east-1-808640880671/scikitlearn-california-train-from-boto3/source.tar.gz'}},
 'ExecutionRoleArn': 'arn:aws:iam::808640880671:role/service-role/AmazonSageMaker-ExecutionRole-20220503T130588',
 'CreationTime': datetime.datetime(2022, 5, 4, 13, 41, 46, 670000, tzinfo=tzlocal()),
 'ModelArn': 'arn:aws:sagemaker:us-east-1:808640880671:model/sklearn-model-2-2022-05-04-13-41-46',
 'EnableNetworkIsolation': False,
 'ResponseMetadata': {'RequestId': '18a7724b-4655-4767-ba0a-7bf33a9756

## Crear una configuración de punto final a partir del segundo modelo

In [44]:
endpoint_config_2_name = "sklearn-endpoint-config-2-" + datetime.datetime.now().strftime(
    "%Y-%m-%d-%H-%M-%S"
)

endpoint_config_2_response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_2_name,
    ProductionVariants=[
        {
            "VariantName": "AllTrafficVariant",
            "ModelName": model_2_name,
            "InitialInstanceCount": 1,
            "InstanceType": "ml.c5.large",
            "InitialVariantWeight": 1,
        },
    ],
)

endpoint_config_2_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:808640880671:endpoint-config/sklearn-endpoint-config-2-2022-05-04-13-41-52',
 'ResponseMetadata': {'RequestId': 'ed248eb7-c651-4b0d-8197-c8943aca107a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ed248eb7-c651-4b0d-8197-c8943aca107a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '126',
   'date': 'Wed, 04 May 2022 13:41:51 GMT'},
  'RetryAttempts': 0}}

## Actualice el punto final en tiempo real con la configuración del segundo punto final

Esto implementará el nuevo `EndpointConfig` especificado en la solicitud, cambiará a usar el punto final recién creado y luego eliminará los recursos aprovisionados para el punto final usando el `EndpointConfig` anterior (no hay pérdida de disponibilidad).

Puede encontrar más información sobre `update_endpoint` en la [página de documentación de Boto3 SageMaker](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.update_endpoint) .

In [45]:
update_endpoint_response = client.update_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_2_name
)

update_endpoint_response

{'EndpointArn': 'arn:aws:sagemaker:us-east-1:808640880671:endpoint/sklearn-endpoint-2022-05-04-13-29-56',
 'ResponseMetadata': {'RequestId': '9514c767-cf08-4ae9-8fe3-1c0bbbfc2d03',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9514c767-cf08-4ae9-8fe3-1c0bbbfc2d03',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '104',
   'date': 'Wed, 04 May 2022 13:42:03 GMT'},
  'RetryAttempts': 0}}

## Espere a que Endpoint esté listo

Navegando a los puntos finales de SageMaker, en la pestaña "Componentes y registros de SageMaker", verá el punto final en el estado "Actualizando".

In [46]:
describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
print(describe_endpoint_response["EndpointStatus"] + "...")

while describe_endpoint_response["EndpointStatus"] == "Updating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    time.sleep(15)

print(describe_endpoint_response["EndpointStatus"])
describe_endpoint_response

InService...
InService


{'EndpointName': 'sklearn-endpoint-2022-05-04-13-29-56',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:808640880671:endpoint/sklearn-endpoint-2022-05-04-13-29-56',
 'EndpointConfigName': 'sklearn-endpoint-config-2-2022-05-04-13-41-52',
 'ProductionVariants': [{'VariantName': 'AllTrafficVariant',
   'DeployedImages': [{'SpecifiedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3',
     'ResolvedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn@sha256:1b1a557ceb336c12d1f8ade5f0cd79781a8df07dde3128544dfef6679feb261e',
     'ResolutionTime': datetime.datetime(2022, 5, 4, 13, 42, 5, 569000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2022, 5, 4, 13, 29, 57, 80000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 5, 4, 13, 44, 30, 194000, tzinfo=tzlocal()),


### Invocar Endpoint con boto3

In [47]:
runtime = boto3.client("sagemaker-runtime")

In [48]:
# csv serialization
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=testX[data.feature_names].to_csv(header=False, index=False).encode("utf-8"),
    ContentType="text/csv",
)

print(response["Body"].read()[:500])

b'[0.49067607142857145, 0.7042345238095238, 4.791390233333334, 2.398275497835498, 2.4403161904761905, 1.5348933333333332, 2.2082266269841275, 1.6368474206349208, 2.9379210833333333, 4.900203, 1.1210192857142858, 1.9788625000000004, 1.6527335714285711, 1.893759126984127, 2.5468676190476183, 1.69058, 1.8855438095238095, 1.6929411111111112, 1.504462142857143, 0.9678378571428572, 3.756083711904761, 4.036424303174603, 1.4885551587301586, 4.089558314285715, 1.6904919372294374, 0.5535566666666666, 1.6606'


## Limpiar

Los puntos finales deben eliminarse cuando ya no se usan, ya que (según la [página de precios de SageMaker](https://aws.amazon.com/sagemaker/pricing/)) se facturan por tiempo de implementación.

In [49]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '3107cd6b-37ea-4000-8d98-f8f7d180182d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3107cd6b-37ea-4000-8d98-f8f7d180182d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 04 May 2022 13:53:57 GMT'},
  'RetryAttempts': 0}}